In [790]:
from modules.FredAPI import FredAPI
from modules.FredAPI import api_key
#uses Fred API to get data.
import numpy as np
import pandas as pd
import scipy.stats as st


In [783]:
fred=FredAPI(api_key)

In [802]:
yields=fred.get_data('DGS2')
#downloading 2Y data

In [805]:
yields=yields.replace('.',np.nan)
#some of the days that have missing data come out like '.'.
yields=yields.fillna(method='bfill').astype(float)
#using the previous day to fill all nan values

In [806]:
yields.index=pd.to_datetime(yields.index)
#converting the index to datetime

In [807]:
diff=yields.diff()
#daily change in yields

In [808]:
result=(diff.iloc[-1]-diff.mean())/diff.std()
# standard deviation  of last trading day move 

In [811]:
st.norm.cdf(result)
#probability of having a larger daily decline

array([0.00010342])

In [812]:
1/(st.norm.cdf(x=diff.iloc[-1].values[0],loc=diff.mean().values[0],scale=diff.std().values[0]))/365
#the price volatility of today should happen once every 26 years

26.492061986566263

In [834]:
day_biggest_decline_ytd=diff.loc['2023':].idxmin()['DGS2']
day_biggest_decline_ytd
#the rally on the 13th of March is so far the biggest daily decline in yields

Timestamp('2023-03-13 00:00:00')

In [823]:
biggest_decline_ytd=diff.loc['2023':].min().values[0]
biggest_decline_ytd
#the actual decline was 57bps

-0.5699999999999994

In [520]:
diff.sort_values(by='DGS2').head(15)
#top 15 biggest declines in 2Y yields

,DGS2
1980-12-19,-0.84
1987-10-20,-0.84
1981-01-05,-0.82
1980-12-22,-0.81
1980-04-16,-0.75
1982-10-07,-0.75
1981-11-09,-0.69
1980-05-02,-0.66
1980-04-04,-0.66
1981-12-04,-0.65


In [815]:
diff.sort_values(by='DGS2').head(15).index.year.value_counts()
#aside from the 1980s, we haven't seen such aggressive yield declines

1980    7
1981    4
1982    2
1987    1
2023    1
dtype: int64

In [826]:
diff.loc[diff['DGS2']<biggest_decline_ytd]
# as a matter of fact, the last time this happened was the day after black monday

,DGS2
1980-04-04,-0.66
1980-04-16,-0.75
1980-04-28,-0.65
1980-05-02,-0.66
1980-12-19,-0.84
1980-12-22,-0.81
1981-01-05,-0.82
1981-10-09,-0.59
1981-11-09,-0.69
1981-12-04,-0.65


In [827]:
day_biggest_decline_ytd

datetime.date(2023, 3, 13)

In [835]:
diff.rank().loc[day_biggest_decline_ytd,'DGS2']
#the rally on the 13th of March was the 14th largest 

14.0

In [836]:
diff.rank().tail(10)
#during the past week we've had several days be in the top 100 volatile

,DGS2
2023-03-06,9117.0
2023-03-07,11659.0
2023-03-08,10315.5
2023-03-09,323.5
2023-03-10,72.5
2023-03-13,14.0
2023-03-14,11973.0
2023-03-15,87.5
2023-03-16,12056.0
2023-03-17,77.0


In [838]:
last_week=diff.loc[diff.index[-1]-pd.offsets.BusinessDay(4):]
pos=last_week[last_week['DGS2']>0]
neg=last_week[last_week['DGS2']<0]
#separating the data in positive and negative change

fig=go.Figure()
fig.add_trace(go.Bar(marker=dict(color='green'),x=(pos.index).date,y=pos['DGS2']*100,showlegend=False))
fig.add_trace(go.Bar(marker=dict(color='maroon'),x=(neg.index).date,y=neg['DGS2']*100,showlegend=False))
fig.update_layout(
    title = '2Y Daily Volatility')
fig.update_xaxes(
    dtick="D1",
    tickformat="%Y-%m-%d")
fig.show()

In [841]:
cumulative_df=pd.DataFrame()
for num in range(1,7):
    cumulative_df=cumulative_df.append((abs(diff).rolling(window=num).sum().rank(ascending=False).tail(10).sort_values(by='DGS2').iloc[0]))
cumulative_df.columns=['Rank']
cumulative_df['Consecutive Days']=range(1,7)

In [842]:
cumulative_df
# in absolute terms, the move on 13th was the 22 largest single day change.
# however, if we look at continuous volatility for more than a day, all days following contributed to the continuous uncertainty
# therefore, if we look at longer periods of time (2,3,4,5,6 consecutive day volatility), this month is still at the top 35

,Rank,Consecutive Days
2023-03-13,22.0,1
2023-03-13,20.0,2
2023-03-14,33.0,3
2023-03-15,31.0,4
2023-03-16,31.0,5
2023-03-17,18.0,6


In [843]:
#one caveat is that if we have 6 consecutive days of high volatility, the 4 day rank will show up multiple times
#our goal is to consider that as a one time event

In [844]:
saved=abs(yields.diff()).rolling(window=4).sum().rank(ascending=False).sort_values(by='DGS2').sort_index()#.head(40).sort_index()

In [845]:
saved.index=pd.to_datetime(saved.index)

In [846]:
saved['Day 1']=saved.index.shift(freq='D')
saved['Day -1']=saved.index.shift(periods=-1,freq='D')

In [855]:
saved_final=saved[saved['DGS2']<saved.loc[cumulative_df.iloc[3].name,'DGS2']]
saved_final['-1']=saved_final.apply(lambda x: 1 if x.name in saved_final['Day 1'].values else 0 ,axis=1)
saved_final['1']=saved_final.apply(lambda x: 1 if x.name in saved_final['Day -1'].values else 0 ,axis=1)

<ipython-input-855-4a3a3adf705f>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-855-4a3a3adf705f>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [511]:
(saved_final['-1']+saved_final['1']==0).sum()
#the 4-day cumulative volatility is reflected by just 1 day above the March 2023 run

9

In [506]:
(saved_final[(saved_final['-1']+saved_final['1']==1)]).sum()['1']
#that is the beginning or end. that should be considered a one-time event

8.0

In [510]:
(saved_final[(saved_final['-1']+saved_final['1']==1)]).sum()['1']+(saved_final['-1']+saved_final['1']==0).sum()
#all events that were more volatile

17.0

In [515]:
saved_final.index.year.value_counts()
#all events were in the 1980s

1980    11
1981    11
1982     6
1987     2
dtype: int64

In [533]:
weekly=abs(diff).resample('W').mean()
weekly.iloc[-1]
#the average volatility this week was 30.28bps

DGS2    0.3028
Name: 2023-03-19 00:00:00, dtype: float64

In [539]:
weekly.rank(ascending=False).iloc[-1]

DGS2    5.0
Name: 2023-03-19 00:00:00, dtype: float64

In [642]:
weekly_more_vol=weekly[weekly['DGS2']>=weekly.iloc[-1].values[0]]
weekly_more_vol
#the only other 4 instances where the week was more volatile happened in early 1980s

,DGS2
1980-12-28,0.3520
1981-01-11,0.4140
1982-08-22,0.3100
1982-10-10,0.3480
2023-03-19,0.3028


In [547]:
#did those moves happened prior/during recessions?
rec=pd.read_html('https://en.wikipedia.org/wiki/List_of_recessions_in_the_United_States')[2]
rec_dates=rec['Period Range'].str.split('–',expand=True)
rec_dates[1]=rec_dates[1].str.split('[',expand=True)[0].str.strip()
for col in rec_dates:
    rec_dates[col]=pd.to_datetime(rec_dates[col])
rec_dates.columns=['Start','End']

In [617]:
during_rec,days_after,days_prior=[],[],[]
for week in weekly_more_vol.index:
    for start,end in zip(rec_dates['Start'],rec_dates['End']):
        check=0
        if (week>start)&(week<end):
            during_rec.append('Yes')
            days_prior.append(np.nan)
            days_after.append(week-start)
            check=1
            break
        else:
            pass
    if check==0:
        during_rec.append('No')
        special=((week-rec_dates['Start']).dt.days)
        prior=week-rec_dates.iloc[special[special<0].idxmax()]['Start']
        days_prior.append(prior)
        days_after.append(np.nan)
            
            

In [622]:
#weekly_more_vol[['Rec','Prior','After']]=[during_rec,days_prior,days_after]
weekly_more_vol=weekly_more_vol.assign(Rec=during_rec,After=days_after,Prior=days_prior)
weekly_more_vol

,DGS2,Rec,Prior,After
1980-12-28,0.352,No,-185 days,NaT
1981-01-11,0.414,No,-171 days,NaT
1982-08-22,0.310,Yes,NaT,417 days
1982-10-10,0.348,Yes,NaT,466 days


In [619]:
#how did markets react the week after that?

In [856]:
pd.merge(weekly_more_vol,weekly.shift(-1),right_index=True,left_index=True,suffixes=[' Cur W','  Nxt W'])
#volatility is dramatically lower

,DGS2 Cur W,DGS2 Nxt W
1980-12-28,0.3520,0.062
1981-01-11,0.4140,0.262
1982-08-22,0.3100,0.174
1982-10-10,0.3480,0.158
2023-03-19,0.3028,NaN


In [655]:
import plotly.graph_objects as go


In [875]:
rec_need=rec_dates[rec_dates['Start']>diff.index[0]]

In [876]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=diff.index,y=diff['DGS2']))
for start,end in zip(rec_need['Start'],rec_need['End']):
    fig.add_shape(x0=start,x1=end,y0=0,y1=1,yref='paper',fillcolor='grey',layer='below',opacity=0.4)

fig.add_hline(y=diff.loc['2023':,'DGS2'].min(),annotation=dict(text='2023-03-15'))
fig.update_layout(title='Daily Volatility in 2Y')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [879]:
fig=go.Figure()
fig.add_trace(go.Scatter(marker=dict(color=color),x=weekly.index,y=weekly['DGS2']))

for start,end in zip(rec_need['Start'],rec_need['End']):
    fig.add_shape(x0=start,x1=end,y0=0,y1=1,yref='paper',fillcolor='grey',layer='below',opacity=0.4)
fig.add_hline(y=weekly.loc['2023':,'DGS2'].max(),annotation=dict(text=f"{((weekly.loc['2023':,'DGS2'].idxmax()).date()-timedelta(days=7))} / {(weekly.loc['2023':,'DGS2'].idxmax()).date().day}"))
for wk,chng in zip(weekly_more_vol.index,weekly_more_vol['DGS2']):
    fig.add_trace(go.Scatter(x=[wk],y=[chng],marker=dict(color='black')))
fig.update_layout(title='Weekly Volatility in 2Y',showlegend=False)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()